Imports

In [1]:
%matplotlib qtagg
import numpy as np
from scipy import constants
import matplotlib.pyplot as plt
from qutip import * 
import qutip as qt

System parameters

In [2]:
omega = 1.0 # H = omega * sigma_z
alpha = 1 / 2
beta = np.sqrt(3) / 2 # psi = alpha |0> + beta |1>
t_list = np.linspace(0, 100, 100)  # time points
psi0 = alpha * basis(2, 0) + beta * basis(2, 1)  # qobj init state

H = omega * sigmaz()

In [ ]:
def qubit_time_evolution_qutip(hamiltonian, alpha, beta, tlist):
    norm = np.sqrt(abs(alpha)**2 + abs(beta)**2)
    alpha, beta = alpha / norm, beta / norm
    
    initial_state = alpha * basis(2, 0) + beta * basis(2, 1)
    
    result = sesolve(hamiltonian, initial_state, tlist)

    bloch = Bloch()
    bloch.add_states(result.states)
    
    bloch.make_sphere()

    plt.title("Qubit Bloch Sphere")
    plt.show()

qubit_time_evolution_qutip(H, alpha, beta, t_list)
